In [38]:
from random import *
def puissance (a,e,n):
  p = 1
  while e>0:
    if (e % 2 != 0):
      p=(p*a)%n
    a=(a*a)%n
    e=e//2
  return p
#Commenter tout le code
#taille des messages

In [39]:
def test_premier_Fermat(n): # test de Fermat

   if ( (puissance(2,n-1,n)==1) and
      (puissance(3,n-1,n)==1) and
      (puissance(5,n-1,n)==1) and
      (puissance(7,n-1,n)==1) and
      (puissance(11,n-1,n)==1) and
      (puissance(13,n-1,n)==1) ):
      return True; # probablement premier (garantie si n<2^15)
   return False

In [40]:
def temoin_Miller(n, a):
    s = 0
    d = n-1
    while d % 2 == 0:
       d = d//2
    x = puissance(a, d, n)
    if(x == 1 or x == n-1):
        return False
    for i in range(s-1):
        x = puissance(x, 2, n)
        if(x == n-1):
            return False
    return True


In [41]:
# Le GMP recommande k=25 pour avoir une proba d'erreur < 2**-50
def test_premier_Miller_Rabin(n, k):
    if n == 2 or n == 3:
       return True
    if n % 2 == 0:
       return False
    for i in range(k):
        a = randint(2, n-2)
        if(temoin_Miller(n, a)):
            return False
    return True


In [42]:
def pgcd(u, v):
  t = 0
  while (v):
    t = u
    u = v
    v = t % v
  if(u<0):
    return -u
  else:
    return u

In [43]:
def bezout(a, b):

  # On sauvegarde les valeurs de a et b.
   a0 = a
   b0 = b

  # On laisse invariant p*a0 + q*b0 = a et  r*a0 + s*b0 = b.
   p = 1
   q = 0
   r = 0
   s = 1
   c = 1
   quotient = 1
   nouveau_r = 1
   nouveau_s = 1

  # La boucle principale.
   while (b != 0):
    c = a % b
    quotient = a//b
    a = b
    b = c
    nouveau_r = p - (quotient * r)
    nouveau_s = q - (quotient * s)
    p = r
    q = s
    r = nouveau_r
    s = nouveau_s
   if p < 0:
     p = p+b0
   return p  # on n'a besoin que de p #si p est négatif retourne p+phi


In [44]:
#vérifier que le message est plus petit que la clé
def generateurClé(c,b):
  p=randint(c,b)
  q=randint(c,b)
  e=randint(c,b)
  d=randint(c,b)

  while test_premier_Miller_Rabin(p,25)==False:
    p = randint(c,b)
  while test_premier_Miller_Rabin(q,25)==False:
    q = randint(c,b)
  #print("2 nombres probablement premier =",p,q)

  n=p*q
  #print("n=p*q=",n)
  
  phi = (p - 1)*(q - 1)
  #print("phi(n)=",phi)
 
  while 1>=e>=phi or pgcd(e,phi)!=1 :
      e = randint(c,b)
  d= int(bezout(int(e),int(phi)))
  #print("cle publique e=",e)
  #print("cle privée d=", d)
  return e,d,n 

In [45]:
def chiffrer(message,clé,n):
    return puissance(message,clé,n)
chiffrer(2192236325473912,1588230257,166681562145852197)

86771824590129366

In [46]:
def dechiffrer(messageChiffre,clé,n):
    return puissance(messageChiffre,clé,n)
dechiffrer(222066741575,771241,230315914859)

198122477765

In [47]:
def str_to_int(m):
    s=0
    b=1
    for i in range(len(m)):
        s=s+ord(m[i])*b
        b=b*256
    return s

In [48]:
def int_to_str(c):
    s =""
    q,r= divmod(c,256)
    s=s+str(chr(r))
    while q != 0:
        q,r = divmod(q,256)
        s =s +str(chr(r))
    return s 

In [49]:
import hashlib
#Probleme de taille de l'empreinte
def empreinte(message): 
    m=hashlib.md5(repr(message).encode())
    return m.hexdigest()[:5]

a=882389747506310411
b=882389747506310411
c=empreinte(a)
print(c)
print(empreinte(b))
print(str_to_int(c))

3f618
3f618
241343817267


In [50]:
def envoieAuthentifie(message,clePrivee,clePublique):
    #Verifie si le message est un string, si oui le convertit en int
    if(isinstance(message,str)):
        message=str_to_int(message)
    #calcule de l'empreinte du message
    e = empreinte(message)
    e=str_to_int(e)
    print("Empreinte: "+str(e))
    
    #Chiffrage de l'empreinte avce la clé privée de l'envoyeur
    c = chiffrer(e,clePrivee[0],clePrivee[1])
    print("EmpreinteChiffré: "+str(c))
    
    #Concaténation du message et de l'empreinte chiffrée
    concatene = int(str(message)+str(c))
    print("Message + empreinte: "+str(concatene))
    #Taille de l'empreinte
    size= len(str(c))
    
    #Envoie du message finale chiffrée avec la clé publique du receveur
    return chiffrer(concatene,clePublique[0],clePublique[1]), size   

In [51]:
def verif_signature(message,clePrivee,clePublique,tailleEmpreinte):

    #Dechiffrer le message signé
    messageD = str(dechiffrer(message,clePrivee[0],clePrivee[1]))
    print("message déchiffré: "+messageD)

    #Separer le message de l'empreinte
    MessageEmpreinteSep=len(messageD)-tailleEmpreinte
    
    messageM=int(messageD[:MessageEmpreinteSep])
    empreinteM=int(messageD[MessageEmpreinteSep:len(messageD)])
    print("Message: "+str(messageM))
    print("Empreinte du message chiffrée: "+str(empreinteM))

    #Déchiffrer l'empreinte avec la clé publique
    EmpreinteD = dechiffrer(empreinteM,clePublique[0],clePublique[1])
    print("ed "+str(EmpreinteD))
    print("ec "+str(str_to_int(empreinte(messageM))))
    #Vérification que l'empreinte déchiffrer correspond à celui du message
    if(str_to_int(empreinte(messageM))==EmpreinteD):
       return messageM #int_to_str() demander si 2 fonctions différente ou 1 seule
    else:
       return False
#verif_signature()
    

In [52]:
#Alice
gA=generateurClé(10**38,10**39)#10**19,10**20
cléPubliqueA= [gA[0],gA[2]]
cléPrivéeA= [gA[1],gA[2]]
print("Clé Publique A: "+str(cléPubliqueA))
print("Clé Privée A: "+str(cléPrivéeA))

Clé Publique A: [322272027337086533995911214483094651161, 485494654261194059842677603902453471316200904816667431805659744195168245621593]
Clé Privée A: [41357710062067451270146068941259111533608240317591000431002368604960026319945, 485494654261194059842677603902453471316200904816667431805659744195168245621593]


In [53]:
#Vérifier que le message est plus petit que la clé
#CA
gCA=generateurClé(10**57,10**60)#10**21,10**23 
cléPubliqueCA= [gCA[0],gCA[2]]
cléPrivéeCA= [gCA[1],gCA[2]]
print("Clé Publique CA: "+str(cléPubliqueCA))
print("Clé Privée CA: "+str(cléPrivéeCA))

Clé Publique CA: [603725442022312414303082991695273136783208623326465588751487, 445328018864501369356618099472372760327218340048999438398586826469100476775127760200790616539264477273235023263421180641]
Clé Privée CA: [257181452137356564388568397789288491510000941017480369930032625223882238657902552965295560438249629578346628481905706423, 445328018864501369356618099472372760327218340048999438398586826469100476775127760200790616539264477273235023263421180641]


In [54]:
#Envoie le message au CA
messageA=envoieAuthentifie(int(cléPubliqueA[0]),cléPrivéeA,cléPubliqueCA)
print("Message de Alice au CA: "+ str(messageA))

Empreinte: 241481103666
EmpreinteChiffré: 104478631139892481258048514728982802292864289553882876855246577466126642225477
Message + empreinte: 322272027337086533995911214483094651161104478631139892481258048514728982802292864289553882876855246577466126642225477
Message de Alice au CA: (169080153156951824255386544524978881459496085111885340643378709714303137541406353606029636984630159922816920276059249795, 78)


In [55]:
#Vérifie la signature 10**39 
#if 
verif=verif_signature(messageA[0],cléPrivéeCA,cléPubliqueA,messageA[1])
assert(verif==cléPubliqueA[0])
CAClePubliqeA=chiffrer(cléPubliqueA[0],cléPrivéeCA[0],cléPrivéeCA[1])
print("Certificat de Alice: "+str(CAClePubliqeA))

message déchiffré: 322272027337086533995911214483094651161104478631139892481258048514728982802292864289553882876855246577466126642225477
Message: 322272027337086533995911214483094651161
Empreinte du message chiffrée: 104478631139892481258048514728982802292864289553882876855246577466126642225477
ed 241481103666
ec 241481103666
Certificat de Alice: 20487599057446613434801010156633529429189377550816605154968485221269919850277852237381722923506116078917339815143513951


In [56]:
#Bob vérifie le certificat d'alice
print("Vérification clé publique Alice: "+str(dechiffrer(CAClePubliqeA,cléPubliqueCA[0],cléPubliqueCA[1])))

Vérification clé publique Alice: 322272027337086533995911214483094651161


In [57]:
#BOB
gB=generateurClé(10**17,10**18)#10**6,10**7
cléPubliqueB= [gB[0],gB[2]]
cléPrivéeB= [gB[1],gB[2]]
print("Clé Publique B: "+str(cléPubliqueB))
print("Clé Privée B: "+str(cléPrivéeB))

#Choix message taille < 10**6
messagez=23847748
messageCd=envoieAuthentifie(messagez,cléPrivéeB,cléPubliqueA)

print("Le message crypté est")
print(messageCd)

messageDk=verif_signature(messageCd[0],cléPrivéeA,cléPubliqueB,messageCd[1])

print("Le message d'origine est")
print(messagez)

print("Le message décrypté est")
print(messageDk)



Clé Publique B: [818760542595493957, 412388075072938169920792771320925381]
Clé Privée B: [61244230560042971740167805887428593, 412388075072938169920792771320925381]
Empreinte: 207117889891
EmpreinteChiffré: 322523763188051539290424487808058909
Message + empreinte: 23847748322523763188051539290424487808058909
Le message crypté est
(387752216273349729660912821780639730912526937954053498136546445140401124407425, 36)
message déchiffré: 23847748322523763188051539290424487808058909
Message: 23847748
Empreinte du message chiffrée: 322523763188051539290424487808058909
ed 207117889891
ec 207117889891
Le message d'origine est
23847748
Le message décrypté est
23847748
